In [1]:
!pip install pyspellchecker

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import re
import numpy as np
import random
import nltk
import string
from contextlib2 import suppress
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
from spellchecker import SpellChecker

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package punkt to C:\Users\Madhav
[nltk_data]     Polakampalli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Madhav
[nltk_data]     Polakampalli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Madhav
[nltk_data]     Polakampalli\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Madhav
[nltk_data]     Polakampalli\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to C:\Users\Madhav
[nltk_data]     Polakampalli\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Madhav
[nltk_data]     

True

In [4]:
f= open(r"D:\downloads\WhatsApp Chat with newme (4).txt", 'r',errors = 'ignore')
raw_doc=f.read()

In [5]:
raw_doc = raw_doc.lower()

In [6]:
sentence_tokens = nltk.sent_tokenize(raw_doc)
word_tokens = nltk.word_tokenize(raw_doc)

In [7]:
print(sentence_tokens)

["choose your concern below and let's get them sorted!", "ðÿ˜ž where's my order?ðÿ¤” help with return?ðÿ“¦ refund check?ðÿ’¸ cancel my orderðÿ™…â€â™‚ï¸ talk to newme ðÿ™‹â€â™€ï¸\nwhere's my order?ðÿ¤”\noh no!", "let's get to the bottom of this.", "when did you order  it's been a week?", 'ðÿ‘œover 15 days?', 'ðÿ˜•?', "ðÿ—“ï¸ðÿ•µï¸â€â™€ï¸\n\nit's been a week?", 'we know waiting can be tough, but our clothes are worth it!', 'standard delivery takes *10-15 business days.', '* thanks for supporting newme.', 'â¤ï¸\n\nis there anything else that you need my help in?', 'chat with a rep ðÿ’¬\n\n\n8/23/23, 10:28â€¯am - ayusha: hey, i need help with my order 429336\nhey newmeog!', "what's up?", "choose your concern below and let's get them sorted!", "ðÿ˜ž where's my order?ðÿ¤” help with return?ðÿ“¦ refund check?ðÿ’¸ cancel my orderðÿ™…â€â™‚ï¸ talk to newme ðÿ™‹â€â™€ï¸\nwhere's my order?ðÿ¤”\noh no!", "let's get to the bottom of this.", 'when did you order?', "it's been a week?", 'ðÿ‘œover 15 days

In [8]:
def recognize_names_with_nltk(text):
  with suppress(UserWarning):
    sentences = sent_tokenize(text)
    words = [word_tokenize(sent) for sent in sentences]

    tagged_words = [pos_tag(word) for word in words]

    named_entities = []
    for tagged_sentence in tagged_words:
        chunked_sentence = ne_chunk(tagged_sentence, binary=False)
        for subtree in chunked_sentence:
            if type(subtree) == nltk.Tree:
                entity = " ".join([word for word, tag in subtree.leaves()])
                label = subtree.label()
                named_entities.append((entity, label))

    return named_entities

recognize_names_with_nltk(raw_doc)

[]

In [9]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

In [10]:
import re

greet_inputs = re.compile(r'(?i)(hello|hi|whatsup|how are you|hey|good evening|greetings|howdy|salutations)')

def greet(sentence):
  match = greet_inputs.search(sentence)
  if match:
    greeting = random.choice(["Hello", "Hi", "Hey", "Hey there","Greetings","Aloha","Good Afternoon"])
    return greeting.format(user_name="User")
  else:
    return None

In [11]:
import re

greet_inputs = re.compile(r'(?i)(hello|hi|whatsup|how are you|hey|good evening|greetings|howdy|salutations)')

def greet(sentence):
  match = greet_inputs.search(sentence)
  if match:
    greeting = random.choice(["Hello", "Hi", "Hey", "Hey there","Greetings","Aloha","Good Afternoon"])
    return greeting.format(user_name="User")
  else:
    return None

In [12]:
with suppress(UserWarning):
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.metrics.pairwise import cosine_similarity

In [13]:
def response(user_response):
  with suppress(UserWarning):
    bot_response = ""
    TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english')
    tfidf = TfidfVec.fit_transform(sentence_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if (req_tfidf == 0):
        bot_response = bot_response + "I am sorry. Unable to understand you!"
        return bot_response
    else:
        bot_response = bot_response + sentence_tokens[idx]
        return bot_response

In [14]:
spell = SpellChecker()

def correct_typos(text):
    words = text.split()
    corrected_words = []

    for word in words:
        corrected_word = spell.correction(word)
        corrected_words.append(corrected_word)

    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [15]:
# def ask_for_feedback():
#     feedback = input("Bot: How would you rate the conversation on a scale of 1 to 5? (1 being poor, 5 being excellent): ")
#     try:
#         rating = int(feedback)
#         if 1 <= rating <= 5:
#             print(f"Bot: Thank you for your feedback! You rated the conversation {rating}/5.")
#         else:
#             print("Bot: Invalid rating. Please provide a rating between 1 and 5.")
#     except ValueError:
#         print("Bot: Invalid input. Please enter a number between 1 and 5.")


In [16]:
flag= True
print("Hello! I am the learning Bot. Start typing your text after greeting to talk to me. For ending convo type bye")
while(flag == True):
  with suppress(UserWarning):
    user_response = input()
    user_response = user_response.lower()
    user_response = correct_typos(user_response)
    if(user_response !='bye'):
      if(user_response =='thank you' or user_response == 'thanks'):
        flag = True
        print("Bot: You are welcome, can I help you with any thing else?")
      elif(user_response =='yes'):
        flag = True
        print("Elaborate your problem please")
      elif(user_response =='no' or user_response == 'no,thanks'):
        flag = False
        print("Alright")
        ask_for_feedback()
      else:
        if(greet(user_response) != None):
          print('Bot: ' +greet(user_response))
        else:
          sentence_tokens.append(user_response)
          word_tokens = word_tokens + nltk.word_tokenize(user_response)
          final_words = list(set(word_tokens))
          print('Bot: ', end = '')
          print(response(user_response))
          sentence_tokens.remove(user_response)
    else:
      flag = False
      print('Bot: Goodbye!')
      # ask_for_feedback()

Hello! I am the learning Bot. Start typing your text after greeting to talk to me. For ending convo type bye
